In [ ]:
!pip install transformers

     |████████████████████████████████| 573kB 2.7MB/s 
     |████████████████████████████████| 890kB 57.6MB/s 
     |████████████████████████████████| 3.7MB 46.4MB/s 
     |████████████████████████████████| 1.0MB 62.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=a6b1a2d31777d30b8c8d5a724f28d705a9e635392e26b69e49762a43f09629e1
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
import torch
import transformers as ppb
import warnings
from google.colab import files
warnings.filterwarnings('ignore')

# **Training**

In [ ]:
#uploaded = files.upload()
df1= pd.read_csv('bert_one.csv',encoding='ISO-8859-1')

pd.DataFrame.from_records(df1)

df1.head()


,Unnamed: 0,X.1,X,listing_id,id,date,comments,attitude,reduced
0,1,1,1,7309129,318235490,9/3/2018,It's the only Airbnb that actually exceeded ex...,1,It's the only Airbnb that actually exceeded ex...
1,2,2,2,20013255,221361135,12/26/2017,READ other REVIEWS about this place. The host ...,0,READ other REVIEWS about this place. The host ...
2,3,3,3,20121308,406954639,1/31/2019,I never stayed here so why falsely say I did. ...,0,I never stayed here so why falsely say I did. ...
3,4,4,4,21186749,554492523,10/26/2019,This place was nothing like the add pictured a...,0,This place was nothing like the add pictured a...
4,5,5,5,30043382,562519976,11/10/2019,This woman is a SCAM ARTIST! BEWARE. My husban...,0,place her why understand I Now available? is m...


In [ ]:
df1['attitude'].value_counts()

0    377
1    123
Name: attitude, dtype: int64

In [ ]:
df1.shape

(500, 9)

In [ ]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-cased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
tokenized = df1['reduced'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length = 512)))

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(500, 512)

In [ ]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
labels=df1['attitude']

In [ ]:
# hold this for other datasets
lr_clf = LogisticRegression()
lr_clf.fit(features, labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
# spliting test/train
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
# train the regression model
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
# evaluate model
lr_clf.score(test_features, test_labels)

0.952

In [ ]:
test_hat = lr_clf.predict(test_features)

In [ ]:
test_hat.shape

(125,)

In [ ]:
confusion_matrix(test_hat,test_labels)

array([[94,  3],
       [ 3, 25]])

In [ ]:
df1.head()

,Unnamed: 0,X,listing_id,id,date,comments,attitude,reduced
1245,1246,1246,347865,447994540,2019-05-04,NOT GOOD CHOICE!! \r\n\r\nIn one of the rooms ...,0,NOT GOOD CHOICE!! \r\n\r\nIn one of the rooms ...
2139,2140,2140,30312506,421519762,3/9/2019,HORRIBLE EXPERIENCE\r\n\r\nThis experience was...,0,HORRIBLE EXPERIENCE\r\n\r\nThis experience was...
1973,1974,1974,16732266,449375492,2019-05-06,Great and easy location. It will be much bette...,1,Great and easy location. It will be much bette...
888,889,889,3697432,113001215,11/9/16,Jime was very nice and the location was perfec...,0,Jime was very nice and the location was perfec...
957,958,958,26278372,332973729,10/6/18,We had to wait 10-15 minutes before assistant ...,1,We had to wait 10-15 minutes before assistant ...


In [ ]:
df2 = pd.read_csv("bert_two.csv",encoding='ISO-8859-1')

In [ ]:
df2.head()

,Unnamed: 0,X.1,X,listing_id,id,date,comments,attitude,reduced
0,1,501,501,4129487,129161858,1/29/2017,The worst part of my stay was when I was about...,0,The worst part of my stay was when I was about...
1,2,502,502,4349358,165029530,6/30/2017,Probably a great value and place to stay in th...,1,Probably a great value and place to stay in th...
2,3,503,503,9617219,58363829,1/2/2016,My son & I drove from Houston TX to NYC and st...,1,My son & I drove from Houston TX to NYC and st...
3,4,504,504,15481609,353789122,11/29/2018,Sadly this was a very bad experience. I??ve g...,0,Sadly this was a very bad experience. I??ve g...
4,5,505,505,18312612,512645481,8/18/2019,DO NOT WASTE YOUR MONEY\r\n\r\nYOU WILL BE DIS...,0,DO NOT WASTE YOUR MONEY\r\n\r\nYOU WILL BE DIS...


In [ ]:
tokenized_one = df2['reduced'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length = 512)))
max_len = 0
for i in tokenized_one.values:
    if len(i) > max_len:
        max_len = len(i)

padded_one = np.array([i + [0]*(max_len-len(i)) for i in tokenized_one.values])
attention_mask_one = np.where(padded_one != 0, 1, 0)
attention_mask_one.shape

(500, 512)

In [ ]:
input_ids = torch.tensor(padded_one)  
attention_mask = torch.tensor(attention_mask_one)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
labels=df2['attitude']

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
lr_clf.score(test_features, test_labels)

0.808

In [ ]:
test_hat = lr_clf.predict(test_features)

In [ ]:
confusion_matrix(test_hat,test_labels)

array([[81, 16],
       [ 8, 20]])

In [ ]:
df3 = pd.read_csv("bert_three.csv",encoding='ISO-8859-1')
df3.head()

,Unnamed: 0,X.1,X,listing_id,id,date,comments,attitude,reduced
0,1,1001,1001,3802538,203743173,10/15/17,"Overall, the stay corresponded to my expectati...",1,"Overall, the stay corresponded to my expectati..."
1,2,1002,1002,7902556,454114730,5/17/19,My worst airbnb experience yet. Dirty bathroom...,0,My worst airbnb experience yet. Dirty bathroom...
2,3,1003,1003,8250457,96609355,8/23/16,Exactly as pictured. Killer neighborhood. Shab...,1,Exactly as pictured. Killer neighborhood. Shab...
3,4,1004,1004,14713599,123090488,12/27/16,My ios airbnb app somehow didnât clearly dif...,0,My ios airbnb app somehow didnât clearly dif...
4,5,1005,1005,24282429,281185315,6/24/18,"Jowelle, we remain very disappointed that your...",0,"Jowelle, we remain very disappointed that your..."


In [ ]:
tokenized_one = df3['reduced'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length = 512)))
max_len = 0
for i in tokenized_one.values:
    if len(i) > max_len:
        max_len = len(i)

padded_one = np.array([i + [0]*(max_len-len(i)) for i in tokenized_one.values])
attention_mask_one = np.where(padded_one != 0, 1, 0)
attention_mask_one.shape

(500, 512)

In [ ]:
input_ids = torch.tensor(padded_one)  
attention_mask = torch.tensor(attention_mask_one)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
labels=df3['attitude']

In [ ]:
three_hat = lr_clf.predict(features)

In [ ]:
df3['group'] = three_hat

In [ ]:
df3['Match'] = np.where(df3['group'] == df3['attitude'], 1,0)
Total = df3['Match'].sum()
print (Total/500)

0.812


In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
lr_clf.score(test_features, test_labels)


0.864

In [ ]:
test_hat = lr_clf.predict(test_features)
confusion_matrix(test_hat,test_labels)

array([[75, 15],
       [ 2, 33]])

In [ ]:
df4 = pd.read_csv("bert_four.csv",encoding='ISO-8859-1')

In [ ]:
tokenized_one = df4['reduced'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length = 512)))
max_len = 0
for i in tokenized_one.values:
    if len(i) > max_len:
        max_len = len(i)

padded_one = np.array([i + [0]*(max_len-len(i)) for i in tokenized_one.values])
attention_mask_one = np.where(padded_one != 0, 1, 0)
attention_mask_one.shape

(500, 512)

In [ ]:
input_ids = torch.tensor(padded_one)  
attention_mask = torch.tensor(attention_mask_one)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
labels=df4['attitude']

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
lr_clf.score(test_features, test_labels)

0.824

In [ ]:
test_hat = lr_clf.predict(test_features)
confusion_matrix(test_hat,test_labels)

array([[76,  7],
       [ 7, 35]])

In [ ]:
lr_clf.fit(test_features,test_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
df5 = pd.read_csv("bert_five.csv",encoding='ISO-8859-1')


In [ ]:
tokenized_one = df5['reduced'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length = 512)))
max_len = 0
for i in tokenized_one.values:
    if len(i) > max_len:
        max_len = len(i)

padded_one = np.array([i + [0]*(max_len-len(i)) for i in tokenized_one.values])
attention_mask_one = np.where(padded_one != 0, 1, 0)
attention_mask_one.shape

(377, 512)

In [ ]:
input_ids = torch.tensor(padded_one)  
attention_mask = torch.tensor(attention_mask_one)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
labels=df5['attitude']

In [ ]:
five_hat = lr_clf.predict(features)
df5['group'] = five_hat

In [ ]:
df5['Match'] = np.where(df5['group'] == df5['attitude'], 1,0)
Total = df5['Match'].sum()
print (Total/377)

0.8275862068965517


In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
lr_clf.score(test_features, test_labels)

0.7894736842105263

In [ ]:
test_hat = lr_clf.predict(test_features)
confusion_matrix(test_hat,test_labels)

array([[48, 14],
       [ 6, 27]])

In [ ]:
lr_clf.fit(test_features,test_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

# **Labelling Section**

In [ ]:
#@title
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
#@title
model_save_name = 'bert_classifier.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model.state_dict(), path)

In [ ]:
df_final1 = pd.read_csv("bert_tbt.csv",nrows = 500)

In [ ]:
df_final1.head()

,Unnamed: 0,listing_id,date,comments,id,label
0,1,12937,2017-10-13,The listing made it sound like it was a privat...,12937,en
1,2,3486103,2017-06-08,Tony's communication before and during our sta...,3486103,en
2,3,13771529,2019-08-09,"Hey guys, so I this was my first booking and e...",13771529,en
3,4,15890547,2018-11-11,Very clean room. Next to highway but not too ...,15890547,en
4,5,17252229,2019-06-16,"nice flat, well connected with line C. noisy n...",17252229,en


In [ ]:
tokenized_one = df_final1['comments'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length = 512)))
max_len = 0
for i in tokenized_one.values:
    if len(i) > max_len:
        max_len = len(i)

padded_one = np.array([i + [0]*(max_len-len(i)) for i in tokenized_one.values])
attention_mask_one = np.where(padded_one != 0, 1, 0)
attention_mask_one.shape

(500, 512)

In [ ]:
input_ids = torch.tensor(padded_one)  
attention_mask = torch.tensor(attention_mask_one)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
prediction = lr_clf.predict(features)
df_final1['prediction'] = prediction

In [ ]:
df_final1.tail()

,Unnamed: 0,listing_id,date,comments,id,label,prediction
495,496,10358093,2018-10-03,Ken responded very quickly to our messages. On...,10358093,en,0
496,497,12669490,2019-08-11,The only issue I had was more so with the neig...,12669490,en,0
497,498,14429957,2018-08-08,The bed and pillows are incredibly uncomfortab...,14429957,en,1
498,499,14988596,2018-11-14,Good location and a good size for a couple who...,14988596,en,0
499,500,15115382,2018-12-13,Marks place was exactly as shown in the pictur...,15115382,en,1


In [ ]:
df_final1.to_csv('bert_1.csv', encoding='utf-8')

In [ ]:
df_final2 = pd.read_csv("bert_tbt.csv",skiprows = range(1,501),nrows = 500)
df_final2.head()

,Unnamed: 0,listing_id,date,comments,id,label
0,501,16324410,2019-07-08,Place was ok. Did not have any bad experience ...,16324410,en
1,502,16478472,2018-10-02,nice (Website hidden by Airbnb) the window and...,16478472,en
2,503,16771876,2019-09-02,"The apartment is very nice, and clean. The apa...",16771876,en
3,504,19011527,2019-11-20,We left after 3 days because the bath was so d...,19011527,en
4,505,20151455,2018-03-20,It’s a small room but if you are not the type ...,20151455,en


In [ ]:
tokenized_one = df_final2['comments'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length = 512)))
max_len = 0
for i in tokenized_one.values:
    if len(i) > max_len:
        max_len = len(i)

padded_one = np.array([i + [0]*(max_len-len(i)) for i in tokenized_one.values])
attention_mask_one = np.where(padded_one != 0, 1, 0)
attention_mask_one.shape

(500, 512)

In [ ]:
input_ids = torch.tensor(padded_one)  
attention_mask = torch.tensor(attention_mask_one)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
prediction = lr_clf.predict(features)
df_final2['prediction'] = prediction
df_final2.to_csv('bert_2.csv', encoding='utf-8')

In [ ]:
df_final3 = pd.read_csv("bert_tbt.csv",skiprows = range(1,1001),nrows = 500)
df_final3.head()

,Unnamed: 0,listing_id,date,comments,id,label
0,1001,21930306,2019-07-04,Seeranie is very thoughtful and responded quic...,21930306,en
1,1002,24472398,2018-09-19,Chandtisse's place is located in a quiet neigh...,24472398,en
2,1003,26939557,2018-10-13,"It was OK, but very minimal efforts could have...",26939557,en
3,1004,27678634,2019-06-04,Easy access to the metro. Guy is super quick a...,27678634,en
4,1005,28430639,2019-02-26,Yoo Mi was very responsive as a host but her l...,28430639,en


In [ ]:
tokenized_one = df_final3['comments'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length = 512)))
max_len = 0
for i in tokenized_one.values:
    if len(i) > max_len:
        max_len = len(i)

padded_one = np.array([i + [0]*(max_len-len(i)) for i in tokenized_one.values])
attention_mask_one = np.where(padded_one != 0, 1, 0)
attention_mask_one.shape

(500, 512)

In [ ]:
input_ids = torch.tensor(padded_one)  
attention_mask = torch.tensor(attention_mask_one)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
prediction = lr_clf.predict(features)
df_final3['prediction'] = prediction
df_final3.to_csv('bert_3.csv', encoding='utf-8')

In [ ]:
df_final4 = pd.read_csv("bert_tbt.csv",skiprows = range(1,1501),nrows = 500)
df_final4.head()

,Unnamed: 0,listing_id,date,comments,id,label
0,1501,33944331,2019-10-06,"Good location, but beware of the vehicle noise...",33944331,en
1,1502,37833505,2019-09-16,Corinna's place has the BEST location! The roo...,37833505,en
2,1503,39277490,2019-11-24,Overall nice place for the price.4 blocks from...,39277490,en
3,1504,39630152,2019-11-17,No television is an issue here. Place is in g...,39630152,en
4,1505,495249,2015-10-08,"Biren's home is really very comfortable, his m...",495249,en


In [ ]:
tokenized_one = df_final4['comments'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length = 512)))
max_len = 0
for i in tokenized_one.values:
    if len(i) > max_len:
        max_len = len(i)

padded_one = np.array([i + [0]*(max_len-len(i)) for i in tokenized_one.values])
attention_mask_one = np.where(padded_one != 0, 1, 0)
attention_mask_one.shape

(500, 512)

In [ ]:
input_ids = torch.tensor(padded_one)  
attention_mask = torch.tensor(attention_mask_one)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
prediction = lr_clf.predict(features)
df_final4['prediction'] = prediction
df_final4.to_csv('bert_4.csv', encoding='utf-8')

In [ ]:
df_final5 = pd.read_csv("bert_tbt.csv",skiprows = range(1,2001),nrows = 500)
df_final5.head()

,Unnamed: 0,listing_id,date,comments,id,label
0,2001,31856192,2019-10-02,"Super close to LGA, nice TV with cable and Car...",31856192,en
1,2002,39024416,2019-10-24,Carlos's place is a good value. \r\n\r\nPros: ...,39024416,en
2,2003,764753,2017-10-09,We were in the appartment from the 2nd to the ...,764753,en
3,2004,2977232,2016-01-08,Alesandra is so nice! I only stay one night in...,2977232,en
4,2005,5748301,2017-04-29,Alberto is super nice. We check in late and Al...,5748301,en


In [ ]:
tokenized_one = df_final5['comments'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length = 512)))
max_len = 0
for i in tokenized_one.values:
    if len(i) > max_len:
        max_len = len(i)

padded_one = np.array([i + [0]*(max_len-len(i)) for i in tokenized_one.values])
attention_mask_one = np.where(padded_one != 0, 1, 0)
attention_mask_one.shape

(500, 512)

In [ ]:
input_ids = torch.tensor(padded_one)  
attention_mask = torch.tensor(attention_mask_one)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
prediction = lr_clf.predict(features)
df_final5['prediction'] = prediction
df_final5.to_csv('bert_5.csv', encoding='utf-8')

In [ ]:
df_final6 = pd.read_csv("bert_tbt.csv",skiprows = range(1,2501),nrows = 500)
df_final6.head()

,Unnamed: 0,listing_id,date,comments,id,label
0,2501,15706863,2018-12-28,"Gretchen’s place is cozy, home away from home ...",15706863,en
1,2502,16140364,2017-08-04,My experience here was not a very good one. Th...,16140364,en
2,2503,19212378,2018-07-07,It was a bad experience. What she displayed on...,19212378,en
3,2504,19439682,2019-12-01,I liked how the place looks like but there wer...,19439682,en
4,2505,21946495,2019-08-05,You will be advised to keep the noise down and...,21946495,en


In [ ]:
tokenized_one = df_final6['comments'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length = 512)))
max_len = 0
for i in tokenized_one.values:
    if len(i) > max_len:
        max_len = len(i)

padded_one = np.array([i + [0]*(max_len-len(i)) for i in tokenized_one.values])
attention_mask_one = np.where(padded_one != 0, 1, 0)
attention_mask_one.shape

(500, 512)

In [ ]:
input_ids = torch.tensor(padded_one)  
attention_mask = torch.tensor(attention_mask_one)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
prediction = lr_clf.predict(features)
df_final6['prediction'] = prediction
df_final6.to_csv('bert_6.csv', encoding='utf-8')

In [ ]:
df_final7 = pd.read_csv("bert_tbt.csv",skiprows = range(1,3001),nrows = 500)
df_final7.head()

,Unnamed: 0,listing_id,date,comments,id,label
0,3001,3539882,2015-12-31,The previous reviews for this listing gave me ...,3539882,en
1,3002,9108196,2016-05-23,"Host called early on the day to now my ETA, sh...",9108196,en
2,3003,9979094,2017-12-02,"Not a bad place to stay for a few days. Nice,...",9979094,en
3,3004,10088636,2016-09-16,"Victor was a very nice host, he always was Che...",10088636,en
4,3005,10812027,2019-09-27,Serafima's airbnb in located in living part of...,10812027,en


In [ ]:
tokenized_one = df_final7['comments'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length = 512)))
max_len = 0
for i in tokenized_one.values:
    if len(i) > max_len:
        max_len = len(i)

padded_one = np.array([i + [0]*(max_len-len(i)) for i in tokenized_one.values])
attention_mask_one = np.where(padded_one != 0, 1, 0)
attention_mask_one.shape

(500, 512)

In [ ]:
input_ids = torch.tensor(padded_one)  
attention_mask = torch.tensor(attention_mask_one)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
prediction = lr_clf.predict(features)
df_final7['prediction'] = prediction
df_final7.to_csv('bert_7.csv', encoding='utf-8')

In [ ]:
df_final8 = pd.read_csv("bert_tbt.csv",skiprows = range(1,3501),nrows = 500)
df_final8.head()

,Unnamed: 0,listing_id,date,comments,id,label
0,3501,26884369,2018-09-15,"the worst experience in an Air b & b, the plac...",26884369,en
1,3502,36205697,2019-09-18,This is a very new construction with very well...,36205697,en
2,3503,1349237,2015-06-05,Very proper and clean appartement. Large bed....,1349237,en
3,3504,3924820,2016-05-08,Marlana's apartment is a really nice place. Ma...,3924820,en
4,3505,4454538,2015-07-12,The apartment is flawless but still my friend ...,4454538,en


In [ ]:
tokenized_one = df_final8['comments'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length = 512)))
max_len = 0
for i in tokenized_one.values:
    if len(i) > max_len:
        max_len = len(i)

padded_one = np.array([i + [0]*(max_len-len(i)) for i in tokenized_one.values])
attention_mask_one = np.where(padded_one != 0, 1, 0)
attention_mask_one.shape

(500, 512)

In [ ]:
input_ids = torch.tensor(padded_one)  
attention_mask = torch.tensor(attention_mask_one)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
prediction = lr_clf.predict(features)
df_final8['prediction'] = prediction
df_final8.to_csv('bert_8.csv', encoding='utf-8')

In [ ]:
df_final9 = pd.read_csv("bert_tbt.csv",skiprows = range(1,4001),nrows = 500)
df_final9.head()

,Unnamed: 0,listing_id,date,comments,id,label
0,4001,57874,2015-07-24,My partner and I are not able to describe this...,57874,en
1,4002,957002,2014-07-16,Sabrina is a curious host. \r\nFirst she is no...,957002,en
2,4003,1533652,2015-01-05,The studio is located in a quiet neighborhood ...,1533652,en
3,4004,4218129,2015-10-02,We were made to feel very welcome on our arriv...,4218129,en
4,4005,4538019,2019-01-02,The host doesn’t speak good English. They talk...,4538019,en


In [ ]:
tokenized_one = df_final9['comments'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length = 512)))
max_len = 0
for i in tokenized_one.values:
    if len(i) > max_len:
        max_len = len(i)

padded_one = np.array([i + [0]*(max_len-len(i)) for i in tokenized_one.values])
attention_mask_one = np.where(padded_one != 0, 1, 0)
attention_mask_one.shape

(500, 512)

In [ ]:
input_ids = torch.tensor(padded_one)  
attention_mask = torch.tensor(attention_mask_one)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
prediction = lr_clf.predict(features)
df_final9['prediction'] = prediction
df_final9.to_csv('bert_9.csv', encoding='utf-8')

In [ ]:
df_final10 = pd.read_csv("bert_tbt.csv",skiprows = range(1,4501),nrows = 500)
df_final10.head()

,Unnamed: 0,listing_id,date,comments,id,label
0,4501,33151516,2019-06-04,It is located between Manhattan and LaGuardia ...,33151516,en
1,4502,2919578,2018-11-30,It located right beside the Parkside Ave Q tra...,2919578,en
2,4503,6327222,2015-05-26,We stayed at the apartment for 6 days. It is a...,6327222,en
3,4504,7125107,2016-05-22,I stayed at Maya's place for one night and it ...,7125107,en
4,4505,10368714,2019-08-20,This place is sufficient if you need a cheap p...,10368714,en


In [ ]:
tokenized_one = df_final10['comments'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length = 512)))
max_len = 0
for i in tokenized_one.values:
    if len(i) > max_len:
        max_len = len(i)

padded_one = np.array([i + [0]*(max_len-len(i)) for i in tokenized_one.values])
attention_mask_one = np.where(padded_one != 0, 1, 0)
attention_mask_one.shape

(500, 512)

In [ ]:
input_ids = torch.tensor(padded_one)  
attention_mask = torch.tensor(attention_mask_one)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
prediction = lr_clf.predict(features)
df_final10['prediction'] = prediction
df_final10.to_csv('bert_10.csv', encoding='utf-8')

In [ ]:
df_final11 = pd.read_csv("bert_tbt.csv",skiprows = range(1,5001),nrows = 500)
df_final11.head()

,Unnamed: 0,listing_id,date,comments,id,label
0,5001,587698,2019-10-04,Jeffrey is a terrific host. You can't go wron...,587698,en
1,5002,1066779,2019-02-15,The location is incredible but the building is...,1066779,en
2,5003,1621235,2019-09-07,Good spot near metro station. Room is comforta...,1621235,en
3,5004,9902314,2016-01-02,Steph provided very detailed instructions on w...,9902314,en
4,5005,15001872,2017-07-13,Donna is terrific!! My husband really enjoyed ...,15001872,en


In [ ]:
tokenized_one = df_final11['comments'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length = 512)))
max_len = 0
for i in tokenized_one.values:
    if len(i) > max_len:
        max_len = len(i)

padded_one = np.array([i + [0]*(max_len-len(i)) for i in tokenized_one.values])
attention_mask_one = np.where(padded_one != 0, 1, 0)
attention_mask_one.shape

(500, 512)

In [ ]:
input_ids = torch.tensor(padded_one)  
attention_mask = torch.tensor(attention_mask_one)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
prediction = lr_clf.predict(features)
df_final11['prediction'] = prediction
df_final11.to_csv('bert_11.csv', encoding='utf-8')

In [ ]:
df_final12 = pd.read_csv("bert_tbt.csv",skiprows = range(1,5501),nrows = 500)
df_final12.head()

,Unnamed: 0,listing_id,date,comments,id,label
0,5501,13246804,2019-10-26,Grace is the best host that I ever met .Her ho...,13246804,en
1,5502,17137626,2019-07-14,Dana and Justin were very sweet. My flight cam...,17137626,en
2,5503,20529328,2019-09-09,"Everything was fine, except of mouse in the room",20529328,en
3,5504,21473314,2019-03-06,Good location if you're just sleeping there......,21473314,en
4,5505,22905091,2019-05-08,"It has brilliant location in Manhattan, but th...",22905091,en


In [ ]:
tokenized_one = df_final12['comments'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length = 512)))
max_len = 0
for i in tokenized_one.values:
    if len(i) > max_len:
        max_len = len(i)

padded_one = np.array([i + [0]*(max_len-len(i)) for i in tokenized_one.values])
attention_mask_one = np.where(padded_one != 0, 1, 0)
attention_mask_one.shape

(500, 512)

In [ ]:
input_ids = torch.tensor(padded_one)  
attention_mask = torch.tensor(attention_mask_one)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
prediction = lr_clf.predict(features)
df_final12['prediction'] = prediction
df_final12.to_csv('bert_12.csv', encoding='utf-8')

In [ ]:
df_final13 = pd.read_csv("bert_tbt.csv",skiprows = range(1,6001),nrows = 500)
df_final13.head()

,Unnamed: 0,listing_id,date,comments,id,label
0,6001,22345271,2019-11-05,This place is exactly as advertised and served...,22345271,en
1,6002,23256946,2019-01-17,Sydey's place is extremely go to get to and fr...,23256946,en
2,6003,23703345,2018-05-28,I’ve used Airbnb domestically and abroad and I...,23703345,en
3,6004,25417007,2018-11-25,It’s nice room but the room is near by the roa...,25417007,en
4,6005,28583186,2019-05-01,A real good place and nice apartment a few min...,28583186,en


In [ ]:
tokenized_one = df_final13['comments'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length = 512)))
max_len = 0
for i in tokenized_one.values:
    if len(i) > max_len:
        max_len = len(i)

padded_one = np.array([i + [0]*(max_len-len(i)) for i in tokenized_one.values])
attention_mask_one = np.where(padded_one != 0, 1, 0)
attention_mask_one.shape

(500, 512)

In [ ]:
input_ids = torch.tensor(padded_one)  
attention_mask = torch.tensor(attention_mask_one)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
prediction = lr_clf.predict(features)
df_final13['prediction'] = prediction
df_final13.to_csv('bert_13.csv', encoding='utf-8')

In [ ]:
df_final14 = pd.read_csv("bert_tbt.csv",skiprows = range(1,6501),nrows = 500)
df_final14.head()

,Unnamed: 0,listing_id,date,comments,id,label
0,6501,25838887,2019-05-14,"Convenient location; clean, simple room; commu...",25838887,en
1,6502,30068829,2019-07-23,Breakfast is served where you check in. Our r...,30068829,en
2,6503,30741302,2019-09-28,"The location is convenient, felt safe, & has g...",30741302,en
3,6504,460036,2015-01-01,We appreciate that Jan was able to accommodate...,460036,en
4,6505,860163,2018-03-03,"This is a true bed and breakfast, a single fam...",860163,en


In [ ]:
tokenized_one = df_final14['comments'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length = 512)))
max_len = 0
for i in tokenized_one.values:
    if len(i) > max_len:
        max_len = len(i)

padded_one = np.array([i + [0]*(max_len-len(i)) for i in tokenized_one.values])
attention_mask_one = np.where(padded_one != 0, 1, 0)
attention_mask_one.shape

(500, 512)

In [ ]:
input_ids = torch.tensor(padded_one)  
attention_mask = torch.tensor(attention_mask_one)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
prediction = lr_clf.predict(features)
df_final14['prediction'] = prediction
df_final14.to_csv('bert_14.csv', encoding='utf-8')

In [ ]:
df_final15 = pd.read_csv("bert_tbt.csv",skiprows = range(1,7001),nrows = 500)
df_final15.head()

,Unnamed: 0,listing_id,date,comments,id,label
0,7001,7770735,2016-05-23,Staying with Esteban and his family was an hon...,7770735,en
1,7002,9786398,2017-03-17,Chelsea and Todd were so nice and laid back. ...,9786398,en
2,7003,19641775,2018-09-03,If you’re looking for an Air BnB do not book h...,19641775,en
3,7004,21609205,2018-08-29,Not a bad place overall but I wouldn't stay th...,21609205,en
4,7005,24194478,2019-11-10,"Nice place, Nice host. Jimmy did the right thi...",24194478,en


In [ ]:
tokenized_one = df_final15['comments'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length = 512)))
max_len = 0
for i in tokenized_one.values:
    if len(i) > max_len:
        max_len = len(i)

padded_one = np.array([i + [0]*(max_len-len(i)) for i in tokenized_one.values])
attention_mask_one = np.where(padded_one != 0, 1, 0)
attention_mask_one.shape

(500, 512)

In [ ]:
input_ids = torch.tensor(padded_one)  
attention_mask = torch.tensor(attention_mask_one)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
prediction = lr_clf.predict(features)
df_final15['prediction'] = prediction
df_final15.to_csv('bert_15.csv', encoding='utf-8')

In [ ]:
df_final16 = pd.read_csv("bert_tbt.csv",skiprows = range(1,7501),nrows = 500)
df_final16.head()

,Unnamed: 0,listing_id,date,comments,id,label
0,7501,14262383,2017-04-17,The host was polite and her communication skil...,14262383,en
1,7502,16793649,2018-07-02,The place was in a quiet neighborhood in Brook...,16793649,en
2,7503,17609541,2017-07-14,I was a disappointed by the place. The host ma...,17609541,en
3,7504,22971074,2018-05-11,Danika's place is very charming. During 20 day...,22971074,en
4,7505,23106220,2018-08-27,Very timely responses to my questions and conc...,23106220,en


In [ ]:
tokenized_one = df_final16['comments'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length = 512)))
max_len = 0
for i in tokenized_one.values:
    if len(i) > max_len:
        max_len = len(i)

padded_one = np.array([i + [0]*(max_len-len(i)) for i in tokenized_one.values])
attention_mask_one = np.where(padded_one != 0, 1, 0)
attention_mask_one.shape

(500, 512)

In [ ]:
input_ids = torch.tensor(padded_one)  
attention_mask = torch.tensor(attention_mask_one)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
prediction = lr_clf.predict(features)
df_final16['prediction'] = prediction
df_final16.to_csv('bert_16.csv', encoding='utf-8')

In [ ]:
df_final17 = pd.read_csv("bert_tbt.csv",skiprows = range(1,8001),nrows = 500)
df_final17.head()

,Unnamed: 0,listing_id,date,comments,id,label
0,8001,17779504,2019-05-04,used slippers provided. super loud creaking st...,17779504,en
1,8002,18121314,2019-08-20,Clean and convenient for the L train. About 3...,18121314,en
2,8003,18858761,2018-02-18,"$79 a night, but can’t put a $40 microwave in ...",18858761,en
3,8004,19641775,2019-06-01,We arrived late at around 12:30 am. Jannices’s...,19641775,en
4,8005,20529760,2017-12-04,The place is as described but this place does ...,20529760,en


In [ ]:
tokenized_one = df_final17['comments'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length = 512)))
max_len = 0
for i in tokenized_one.values:
    if len(i) > max_len:
        max_len = len(i)

padded_one = np.array([i + [0]*(max_len-len(i)) for i in tokenized_one.values])
attention_mask_one = np.where(padded_one != 0, 1, 0)
attention_mask_one.shape

(500, 512)

In [ ]:
input_ids = torch.tensor(padded_one)  
attention_mask = torch.tensor(attention_mask_one)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
prediction = lr_clf.predict(features)
df_final17['prediction'] = prediction
df_final17.to_csv('bert_17.csv', encoding='utf-8')

In [ ]:
df_final18 = pd.read_csv("bert_tbt.csv",skiprows = range(1,8501),nrows = 500)
df_final18.head()

,Unnamed: 0,listing_id,date,comments,id,label
0,8501,9394266,2016-07-31,Nothing was too much trouble for Antonia. I en...,9394266,en
1,8502,11676984,2017-08-24,"a very nice, attractively furnished apartment ...",11676984,en
2,8503,20210285,2017-10-24,We felt very welcomed and safe in this air bnb...,20210285,en
3,8504,21289522,2018-01-17,Roland is very nice and welcoming. Even though...,21289522,en
4,8505,21485174,2019-04-20,"Pascale’s place, unlike some hotel rooms, has ...",21485174,en


In [ ]:
tokenized_one = df_final18['comments'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length = 512)))
max_len = 0
for i in tokenized_one.values:
    if len(i) > max_len:
        max_len = len(i)

padded_one = np.array([i + [0]*(max_len-len(i)) for i in tokenized_one.values])
attention_mask_one = np.where(padded_one != 0, 1, 0)
attention_mask_one.shape

(500, 512)

In [ ]:
input_ids = torch.tensor(padded_one)  
attention_mask = torch.tensor(attention_mask_one)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
prediction = lr_clf.predict(features)
df_final18['prediction'] = prediction
df_final18.to_csv('bert_18.csv', encoding='utf-8')

In [ ]:
df_final19 = pd.read_csv("bert_tbt.csv",skiprows = range(1,9001),nrows = 500)
df_final19.head()

,Unnamed: 0,listing_id,date,comments,id,label
0,9001,6787310,2018-08-04,It was very clean and nice room with private b...,6787310,en
1,9002,8889118,2018-12-02,We were very disappointed with this apartment!...,8889118,en
2,9003,9029289,2017-06-19,Marcello and his place are the best... Can't r...,9029289,en
3,9004,10855464,2016-04-12,Very hospitable and very flexible on check in ...,10855464,en
4,9005,10883053,2016-10-29,I arrived and the host was a late and it was a...,10883053,en


In [ ]:
tokenized_one = df_final19['comments'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length = 512)))
max_len = 0
for i in tokenized_one.values:
    if len(i) > max_len:
        max_len = len(i)

padded_one = np.array([i + [0]*(max_len-len(i)) for i in tokenized_one.values])
attention_mask_one = np.where(padded_one != 0, 1, 0)
attention_mask_one.shape

(500, 512)

In [ ]:
input_ids = torch.tensor(padded_one)  
attention_mask = torch.tensor(attention_mask_one)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
prediction = lr_clf.predict(features)
df_final19['prediction'] = prediction
df_final19.to_csv('bert_19.csv', encoding='utf-8')

In [ ]:
df_final20 = pd.read_csv("bert_tbt.csv",skiprows = range(1,9501),nrows = 500)
df_final20.head()

,Unnamed: 0,listing_id,date,comments,id,label
0,9501,5192219,2016-12-18,Very interesting location enjoyed my stay in H...,5192219,en
1,9502,6051044,2017-01-15,Nice and quiet place near subway if you need t...,6051044,en
2,9503,6093352,2017-11-11,I have to tell everyone to avoid this place: ...,6093352,en
3,9504,7376952,2018-10-27,Katherine received us in the best way expected...,7376952,en
4,9505,7453537,2019-10-25,This was by far the best AirBnB experience I’v...,7453537,en


In [ ]:
tokenized_one = df_final20['comments'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length = 512)))
max_len = 0
for i in tokenized_one.values:
    if len(i) > max_len:
        max_len = len(i)

padded_one = np.array([i + [0]*(max_len-len(i)) for i in tokenized_one.values])
attention_mask_one = np.where(padded_one != 0, 1, 0)
attention_mask_one.shape

(500, 512)

In [ ]:
input_ids = torch.tensor(padded_one)  
attention_mask = torch.tensor(attention_mask_one)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
prediction = lr_clf.predict(features)
df_final20['prediction'] = prediction
df_final20.to_csv('bert_20.csv', encoding='utf-8')

In [ ]:
df_final20.tail()

,Unnamed: 0,listing_id,date,comments,id,label,prediction
495,9996,24489174,2019-11-05,Room as advertised. It's very convenient and c...,24489174,en,1
496,9997,24942474,2018-09-20,Very accommodating hosts. A bed would be prefe...,24942474,en,0
497,9998,25161242,2019-02-14,"Can go wrong with this quaint, cozy, clean space.",25161242,en,0
498,9999,27052657,2018-10-08,"As advertised, studio a bit small. Try not to ...",27052657,en,0
499,10000,28131065,2019-06-01,Overall stay was good. Understandable that the...,28131065,en,0


In [ ]:
df_final21 = pd.read_csv("bert_tbt.csv",skiprows = range(1,10001),nrows = 500)
df_final21.head()

,Unnamed: 0,listing_id,date,comments,id,label
0,10001,28548195,2019-06-28,I wouldn't recommend,28548195,en
1,10002,29740171,2019-08-13,The doors DO NOT have a key lock on bedroom or...,29740171,en
2,10003,30592647,2019-05-20,It was a pleasant stay in Jimmy's house! The s...,30592647,en
3,10004,31175570,2019-05-09,"Place was spacious, clean, and stylish. Ten mi...",31175570,en
4,10005,38115292,2019-08-31,After canceling 2 other bookings for these dat...,38115292,en
